In [1]:
import os
import re
from typing import List, Optional
from warnings import warn
import pandas as pd
import numpy as np
from tqdm import tqdm
from shapely.geometry import Polygon
pd.set_option('display.max_columns', None)
tqdm.pandas()

In [2]:
def get_filenames_from_dir(path: str,
                          mandatory_substr: str='',
                          include_subdirs: bool=True,
                          file_extensions: Optional[List[str]]=None):
    """Получает пути к файлам с данными из директории и поддиректорий

    Args:
        path (str): путь к корневому каталогу с файлами
        mandatory_substr (str): обязательная подстрока, которая должна содержаться в имени файла
        include_subdirs (bool, optional):смотреть ли в поддиректориях. Defaults to True.
        file_extensions (str or [str], optional): какие расширения нас интересуют. Defaults to None.

    Returns:
        [str]: список путей к файлам
    """
    files_list = []

    if include_subdirs:
        for root, _, files in os.walk(path):
            files_list += [os.path.join(root, f) for f in files if os.path.isfile(os.path.join(root, f))]
    else:
        files_list = [os.path.join(path, f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

    if file_extensions is not None:
        files_list = [f for f in files_list if f.split('.')[-1] in file_extensions]

    files_list = [f for f in files_list if mandatory_substr in f]

    return files_list

In [3]:
GEODATA_COL = 'GEODATA'
GEODATA_CENTER_COL = 'GEODATA_CENTER'
GEO_COLS = [GEODATA_COL, GEODATA_CENTER_COL]
KAD_NUM_COLS = ['KAD_N', 'KAD_ZU']


def create_geolist(geo_str):
    geo_list = re.sub(r'[^0-9,.]', '', str(geo_str)).split(',')
    if geo_list != []:
        geo_list = [[float(geo_list[i]), float(geo_list[i+1])] for i in range(0, len(geo_list)-1, 2)]
    return geo_list


def prepare_geo(geodata: str) -> str:
    geodata = re.sub(r'[^\[\]0-9,.]', '', str(geodata))
    return geodata.strip(',')


def prepare_kad_num(kad_num: str) -> str:
    kad_num = re.sub(r'[^0-9\:]', '', str(kad_num))
    if (kad_num!='' and kad_num[0]==':'):
        return kad_num[1:]
    return kad_num


def calc_polygon_centroid(coords):
    if not isinstance(coords, list):
        warn('coords is not list')
        return np.nan
    if coords==[]:
        warn('coords is empty')
        return np.nan
    plgn = Polygon(coords)
    return list(plgn.centroid.coords)[0]


def prepare_data_mos_ru(data,
                        geodata_col: str=GEODATA_COL,
                        geodata_center_col: str=GEODATA_CENTER_COL,
                        geo_cols: Optional[List[str]]=None,
                        kad_num_cols: Optional[List[str]]=None):
    if geo_cols is None:
        geo_cols = GEO_COLS
    if kad_num_cols is None:
        kad_num_cols = KAD_NUM_COLS

    data = data.copy()
    data = data.fillna('')
    data.columns = [col.upper() for col in data.columns]

    for col in geo_cols:
        data[col] = data[col].progress_apply(prepare_geo)

    for col in kad_num_cols:
        data[col] = data[col].progress_apply(prepare_kad_num)

    data[geodata_col] = data[geodata_col].progress_apply(create_geolist)
    data[geodata_center_col] = data[geodata_col].progress_apply(calc_polygon_centroid)

    return data

In [4]:
# print(calc_polygon_centroid([[0,0], [1,0], [1,1], [0,1]]),
#       'expect: [.5, .5]')
# print(calc_polygon_centroid([[0,1], [0,-1], [2,0]]),
#       'expect: [2/3, 0]')
# print(calc_polygon_centroid([[0,-1], [0,1], [2,0]]),
#       'expect: [2/3, 0]')
# print(calc_polygon_centroid([[0,0], [1,0], [1.5,0.5], [1,1], [0,1], [-.5,.5]]),
#       'expect: [.5, .5]')

In [5]:
data_mos_ru = pd.read_json('/Users/affernus/Downloads/data-29580-2022-10-20.json',
                            encoding='cp1251').dropna(axis=1, how='all')

In [6]:
prepared_dmr = prepare_data_mos_ru(data_mos_ru)
prepared_dmr['N_FIAS'] = prepared_dmr['N_FIAS'].str.upper()

  0%|          | 0/475401 [00:00<?, ?it/s]/var/folders/7q/8vvk22jj58jd3fvwfg98269m0000gn/T/ipykernel_20979/1766981003.py:31: UserWarning: coords is empty
  warn('coords is empty')
100%|██████████| 475401/475401 [00:19<00:00, 24047.20it/s]


In [7]:
with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
    display(prepared_dmr.head(2))
    print(prepared_dmr.shape)

,GLOBAL_ID,OBJ_TYPE,ONTERRITORYOFMOSCOW,ADDRESS,SIMPLE_ADDRESS,UNOM,P0,P1,P2,P5,P7,L1_TYPE,L1_VALUE,L2_VALUE,L3_VALUE,L4_VALUE,L5_VALUE,ADM_AREA,DISTRICT,NREG,DREG,N_FIAS,D_FIAS,KAD_N,KAD_ZU,KLADR,TDOC,NDOC,DDOC,ADR_TYPE,VID,SOSTAD,STATUS,GEODATA,GEODATA_CENTER,L3_TYPE,L2_TYPE,P4,P6,P3,P90,P91
0,645622141,Здание,да,"Российская Федерация, город Москва, внутригородская территория муниципальный округ Вешняки, Косинская улица, дом 26А","Косинская улица, дом 26А",2102436,Российская Федерация,город Москва,внутригородская территория муниципальный округ Вешняки,муниципальный округ Вешняки,Косинская улица,дом,26А,,,,,Восточный административный округ,муниципальный округ Вешняки,3303053.0,03.08.2004,235212A3-01E8-4CC3-87D5-59F00C83898A,27.02.2012,77:03:0007004:1064,77:03:0007004:6443,77000000000040000,Распоряжение префектуры АО города Москвы,875-В-РП,28.07.2004,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"[[37.8279504545784, 55.7176609928454], [37.8286195050135, 55.7174401927724], [37.8284661997432, 55.7172920310098], [37.8280404992056, 55.7174326150897], [37.8279795383341, 55.7174527408861], [37.8277971493706, 55.7175128293572], [37.8279504545784, 55.7176609928454]]","(37.828208360207746, 55.71747652735648)",,,,,,,
1,645622142,Здание,да,"Российская Федерация, город Москва, внутригородская территория муниципальный округ Басманный, Гороховский переулок, дом 21","Гороховский переулок, дом 21",5783,Российская Федерация,город Москва,внутригородская территория муниципальный округ Басманный,муниципальный округ Басманный,Гороховский переулок,дом,21,,,,,Центральный административный округ,муниципальный округ Басманный,1011856.0,13.07.2005,533D296D-1ECC-49EA-9156-DCFA8E38E4D8,27.02.2012,77:01:0003010:1018,77:01:0003010:4146,77000000000112200,Распоряжение префектуры АО города Москвы,3026-р,04.07.2005,Официальный,Адрес,Зарегистрирован в АР,Внесён в ГКН,"[[37.6682995208392, 55.7662431483298], [37.6683153361192, 55.7662161815465], [37.6682467711738, 55.7662023546956], [37.6682610223573, 55.7661821263411], [37.6681876684064, 55.7661660596898], [37.6681726244854, 55.7661871863933], [37.6681295710992, 55.7661782636006], [37.6681461617712, 55.7661463558389], [37.6680536825827, 55.7661285198332], [37.6680283160622, 55.7661572950872], [37.6678266067447, 55.7661162557855], [37.6677727488124, 55.7661881829886], [37.6677504217352, 55.7661828244461], [37.667709246984, 55.766240362676], [37.6679436471296, 55.766288544657], [37.6679270349883, 55.7663155115827], [37.6680322751771, 55.7663369247134], [37.6680496857547, 55.7663104048687], [37.6682649515806, 55.7663545688389], [37.668302916126, 55.7662916457961], [37.6683314068679, 55.7662484936784], [37.6682995208392, 55.7662431483298]]","(37.668037361143384, 55.766233199632005)",,,,,,,


(475401, 42)


In [8]:
houses_data_files = get_filenames_from_dir(
    '/Users/affernus/Downloads/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/',
    mandatory_substr='csv')
display(houses_data_files)

['/Users/affernus/Downloads/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_2.csv',
 '/Users/affernus/Downloads/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_3.csv',
 '/Users/affernus/Downloads/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_1.csv',
 '/Users/affernus/Downloads/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_4.csv',
 '/Users/affernus/Downloads/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_5.csv',
 '/Users/affernus/Downloads/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_7.csv',
 '/Users/affernus/Downloads/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_6.csv',
 '/Users/affernus/Downloads/Сведения_об_объектах_жилищного_фонда_на_23-10-2022/Сведения по ОЖФ Москва г на 23-10-2022_

In [9]:
houses_data_list = []
for hd in houses_data_files:
    houses_data_list.append(pd.read_csv(hd, delimiter=';', low_memory=False))
houses_data = pd.concat(houses_data_list, ignore_index=True).drop_duplicates().reset_index(drop=True)

In [10]:
houses_data['Глобальный уникальный идентификатор дома по ФИАС'] = houses_data['Глобальный уникальный идентификатор дома по ФИАС'].str.upper()

In [11]:
with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
    display(houses_data.head(5))
    print(houses_data.shape)

,Адрес ОЖФ,Идентификационный код адреса дома в ГИС ЖКХ,Глобальный уникальный идентификатор дома по ФИАС,Код ОКТМО,Способ управления,"ОГРН организации, осуществляющей управление домом","КПП организации, осуществляющей управление домом","Наименование организации, осуществляющей управление домом",Тип дома,Состояние,Общая площадь дома,Жилая площадь в доме,Дом находится в собственности субъекта Российской Федерации и в полном объеме используется в качестве общежития,Дом находится в муниципальной собственности и в полном объеме используется в качестве общежития,Тип общежития,Тип помещения (блока),Дата сноса объекта,Номер помещения (блока),Номер комнаты,Кадастровый номер,Глобальный уникальный идентификатор дома,Глобальный уникальный идентификатор помещения,Глобальный уникальный идентификатор комнаты
0,"108809, Москва г, п. Марушкинское, д. Марушкино, ул. Строителей, д. 3",f93a19c3-ffab-48b5-92c3-d10a49166a72,F93A19C3-FFAB-48B5-92C3-D10A49166A72,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,396.2,NaN,-,-,-,NaN,NaN,NaN,NaN,50:26:0170402:2106,dc41973a-8cdd-4298-b7d7-365cd5b96a09,NaN,NaN
1,"108809, Москва г, п. Марушкинское, д. Марушкино, ул. Строителей, д. 3А",9e595cc4-fa15-4d22-bc17-380b691f47c1,9E595CC4-FA15-4D22-BC17-380B691F47C1,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,124.9,NaN,-,-,-,NaN,NaN,NaN,NaN,нет,407142e2-e27c-4973-b614-7cd939e5a037,NaN,NaN
2,"108809, Москва г, п. Марушкинское, д. Марушкино, ул. Строителей, д. 4",281b5116-2cef-4364-a850-1bb102687bfa,35F6CD9F-509B-4C92-839F-B03F70DE53E5,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,209.3,NaN,-,-,-,NaN,NaN,NaN,NaN,77:18:0170406:50,66eabcec-e02b-408e-9629-d17935b14399,NaN,NaN
3,"108809, Москва г, п. Марушкинское, д. Марушкино, ул. Строителей, д. 4",281b5116-2cef-4364-a850-1bb102687bfa,35F6CD9F-509B-4C92-839F-B03F70DE53E5,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,209.3,NaN,-,-,-,NaN,NaN,NaN,1,NaN,66eabcec-e02b-408e-9629-d17935b14399,NaN,a18b5ed7-98d0-4364-960f-63720f8fee31
4,"108809, Москва г, п. Марушкинское, д. Марушкино, ул. Строителей, д. 6",855583f2-ce42-4ad0-9c9a-cfca34288c7f,10860E59-E7D1-43C2-A19B-1856644E9B3C,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,145.6,NaN,-,-,-,NaN,NaN,NaN,NaN,77:18:0170406:388,505249be-5ff4-4f7f-bfdc-222aab4cae19,NaN,NaN


(5161209, 23)


In [12]:
not_room = houses_data['Номер помещения (блока)'].isna() & houses_data['Номер комнаты'].isna()
houses_data_not_room = houses_data[not_room]
print(houses_data_not_room.shape)

(68318, 23)


In [19]:
display(houses_data_not_room.head())

,Адрес ОЖФ,Идентификационный код адреса дома в ГИС ЖКХ,Глобальный уникальный идентификатор дома по ФИАС,Код ОКТМО,Способ управления,"ОГРН организации, осуществляющей управление домом","КПП организации, осуществляющей управление домом","Наименование организации, осуществляющей управление домом",Тип дома,Состояние,Общая площадь дома,Жилая площадь в доме,Дом находится в собственности субъекта Российской Федерации и в полном объеме используется в качестве общежития,Дом находится в муниципальной собственности и в полном объеме используется в качестве общежития,Тип общежития,Тип помещения (блока),Дата сноса объекта,Номер помещения (блока),Номер комнаты,Кадастровый номер,Глобальный уникальный идентификатор дома,Глобальный уникальный идентификатор помещения,Глобальный уникальный идентификатор комнаты
0,"108809, Москва г, п. Марушкинское, д. Марушкин...",f93a19c3-ffab-48b5-92c3-d10a49166a72,F93A19C3-FFAB-48B5-92C3-D10A49166A72,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,396.2,NaN,-,-,-,NaN,NaN,NaN,NaN,50:26:0170402:2106,dc41973a-8cdd-4298-b7d7-365cd5b96a09,NaN,NaN
1,"108809, Москва г, п. Марушкинское, д. Марушкин...",9e595cc4-fa15-4d22-bc17-380b691f47c1,9E595CC4-FA15-4D22-BC17-380B691F47C1,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,124.9,NaN,-,-,-,NaN,NaN,NaN,NaN,нет,407142e2-e27c-4973-b614-7cd939e5a037,NaN,NaN
2,"108809, Москва г, п. Марушкинское, д. Марушкин...",281b5116-2cef-4364-a850-1bb102687bfa,35F6CD9F-509B-4C92-839F-B03F70DE53E5,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,209.3,NaN,-,-,-,NaN,NaN,NaN,NaN,77:18:0170406:50,66eabcec-e02b-408e-9629-d17935b14399,NaN,NaN
4,"108809, Москва г, п. Марушкинское, д. Марушкин...",855583f2-ce42-4ad0-9c9a-cfca34288c7f,10860E59-E7D1-43C2-A19B-1856644E9B3C,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,145.6,NaN,-,-,-,NaN,NaN,NaN,NaN,77:18:0170406:388,505249be-5ff4-4f7f-bfdc-222aab4cae19,NaN,NaN
5,"108809, Москва г, п. Марушкинское, д. Марушкин...",75f9b36c-5b8e-4c81-9215-b5fc4d462c37,DBC3B094-D0E1-4FC3-A5F7-044B88FC8124,45949000,Не выбран,NaN,NaN,NaN,Жилой,Исправный,224.1,NaN,-,-,-,NaN,NaN,NaN,NaN,50:26:0170402:2279,dae214a7-d984-4c71-9c97-48215a5fa8bf,NaN,NaN


In [22]:
houses_data[houses_data['Жилая площадь в доме'].notna()]

,Адрес ОЖФ,Идентификационный код адреса дома в ГИС ЖКХ,Глобальный уникальный идентификатор дома по ФИАС,Код ОКТМО,Способ управления,"ОГРН организации, осуществляющей управление домом","КПП организации, осуществляющей управление домом","Наименование организации, осуществляющей управление домом",Тип дома,Состояние,Общая площадь дома,Жилая площадь в доме,Дом находится в собственности субъекта Российской Федерации и в полном объеме используется в качестве общежития,Дом находится в муниципальной собственности и в полном объеме используется в качестве общежития,Тип общежития,Тип помещения (блока),Дата сноса объекта,Номер помещения (блока),Номер комнаты,Кадастровый номер,Глобальный уникальный идентификатор дома,Глобальный уникальный идентификатор помещения,Глобальный уникальный идентификатор комнаты
460,"108809, Москва г, п. Толстопальцево, ул. Ленин...",50426927-4bc2-425b-a974-01267c073d61,1B67EFF3-3B5E-4F19-8E8D-367DA5185D6A,45317000,УО,5.137746e+12,772901001.0,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,682.7,384.9,-,-,-,NaN,NaN,NaN,NaN,77:07:0017001:1313,e78f0284-6122-4a5c-8803-7bdbdc6695d5,NaN,NaN
461,"108809, Москва г, п. Толстопальцево, ул. Ленин...",50426927-4bc2-425b-a974-01267c073d61,1B67EFF3-3B5E-4F19-8E8D-367DA5185D6A,45317000,УО,5.137746e+12,772901001.0,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,682.7,384.9,-,-,-,Жилое,NaN,1,NaN,77:07:0017001:1741,e78f0284-6122-4a5c-8803-7bdbdc6695d5,f24e5358-c90b-467b-8597-7d79ab6f9701,NaN
462,"108809, Москва г, п. Толстопальцево, ул. Ленин...",50426927-4bc2-425b-a974-01267c073d61,1B67EFF3-3B5E-4F19-8E8D-367DA5185D6A,45317000,УО,5.137746e+12,772901001.0,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,682.7,384.9,-,-,-,Жилое,NaN,2,NaN,NaN,e78f0284-6122-4a5c-8803-7bdbdc6695d5,3bec370a-3cb5-4a7f-a14e-87523b62a66b,NaN
463,"108809, Москва г, п. Толстопальцево, ул. Ленин...",50426927-4bc2-425b-a974-01267c073d61,1B67EFF3-3B5E-4F19-8E8D-367DA5185D6A,45317000,УО,5.137746e+12,772901001.0,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,682.7,384.9,-,-,-,Жилое,NaN,3,NaN,77:07:0014009:10175,e78f0284-6122-4a5c-8803-7bdbdc6695d5,9dfdfffa-0d0c-4e0e-96cf-0fdf01bb3ed3,NaN
464,"108809, Москва г, п. Толстопальцево, ул. Ленин...",50426927-4bc2-425b-a974-01267c073d61,1B67EFF3-3B5E-4F19-8E8D-367DA5185D6A,45317000,УО,5.137746e+12,772901001.0,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,682.7,384.9,-,-,-,Жилое,NaN,6,NaN,NaN,e78f0284-6122-4a5c-8803-7bdbdc6695d5,ef3045af-cb25-4798-910a-04263aa77e1c,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5161204,"129323, Москва г, проезд. Русанова, д. 31",3b980976-358d-40af-95a6-4a8dbeb0cdbe,3B980976-358D-40AF-95A6-4A8DBEB0CDBE,45361000,УО,1.087746e+12,771601001.0,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,24463.2,16565,-,-,-,Жилое,NaN,282,NaN,77:02:0014011:14496,4615c33c-c2a3-4436-855f-ce966fe8c20d,6047714c-e54f-48bc-9f90-fcb2a5d798a4,NaN
5161205,"129323, Москва г, проезд. Русанова, д. 31",3b980976-358d-40af-95a6-4a8dbeb0cdbe,3B980976-358D-40AF-95A6-4A8DBEB0CDBE,45361000,УО,1.087746e+12,771601001.0,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,24463.2,16565,-,-,-,Жилое,NaN,283,NaN,77:02:0014011:14495,4615c33c-c2a3-4436-855f-ce966fe8c20d,7e662d53-c3f9-4a65-818a-4c4b56149cc9,NaN
5161206,"129323, Москва г, проезд. Русанова, д. 31",3b980976-358d-40af-95a6-4a8dbeb0cdbe,3B980976-358D-40AF-95A6-4A8DBEB0CDBE,45361000,УО,1.087746e+12,771601001.0,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,24463.2,16565,-,-,-,Жилое,NaN,284,NaN,77:02:0014011:14494,4615c33c-c2a3-4436-855f-ce966fe8c20d,3313ce2c-817f-4f9f-b19e-9431f26cd923,NaN
5161207,"129323, Москва г, проезд. Русанова, д. 31",3b980976-358d-40af-95a6-4a8dbeb0cdbe,3B980976-358D-40AF-95A6-4A8DBEB0CDBE,45361000,УО,1.087746e+12,771601001.0,ОБЩЕ

In [21]:
houses_data_not_room[houses_data_not_room['Жилая площадь в доме'].notna()]

,Адрес ОЖФ,Идентификационный код адреса дома в ГИС ЖКХ,Глобальный уникальный идентификатор дома по ФИАС,Код ОКТМО,Способ управления,"ОГРН организации, осуществляющей управление домом","КПП организации, осуществляющей управление домом","Наименование организации, осуществляющей управление домом",Тип дома,Состояние,Общая площадь дома,Жилая площадь в доме,Дом находится в собственности субъекта Российской Федерации и в полном объеме используется в качестве общежития,Дом находится в муниципальной собственности и в полном объеме используется в качестве общежития,Тип общежития,Тип помещения (блока),Дата сноса объекта,Номер помещения (блока),Номер комнаты,Кадастровый номер,Глобальный уникальный идентификатор дома,Глобальный уникальный идентификатор помещения,Глобальный уникальный идентификатор комнаты
460,"108809, Москва г, п. Толстопальцево, ул. Ленин...",50426927-4bc2-425b-a974-01267c073d61,1B67EFF3-3B5E-4F19-8E8D-367DA5185D6A,45317000,УО,5.137746e+12,772901001.0,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,682.7,384.9,-,-,-,NaN,NaN,NaN,NaN,77:07:0017001:1313,e78f0284-6122-4a5c-8803-7bdbdc6695d5,NaN,NaN
471,"108809, Москва г, п. Толстопальцево, ул. Ленин...",235046af-b5b6-4d22-8598-57ba12d62676,D0015BD7-B3A7-4479-89AB-EE44609351DF,45317000,УО,5.137746e+12,772901001.0,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ГОРОДА МО...,Многоквартирный,Исправный,685.6,385.1,-,-,-,NaN,NaN,NaN,NaN,77:07:0017001:1319,c35f6a99-4f7e-458a-a446-fbab99d85a44,NaN,NaN
636,"108809, Москва г, п. Толстопальцево, ул. Осипе...",84d6e440-e28d-4646-bc40-117e68b648b6,0E6C6EEE-F1A9-404C-AC91-BCE7B6DD66A5,45317000,УО,1.127747e+12,775101001.0,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,322.0,212.5,-,-,-,NaN,NaN,NaN,NaN,нет,e94282b0-9be9-4b7e-8165-d68fb68d655f,NaN,NaN
1453,"108810, Москва г, п. Марушкинское, д. Большое ...",c520fd52-e3ad-4b3c-bc0e-62659b8094cd,010DB36D-A2F5-416C-90A6-9E4D612146F2,45949000,УО,1.207700e+12,773001001.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""УК Н...",Многоквартирный,Исправный,796.4,672.1,-,-,-,NaN,NaN,NaN,NaN,77:18:0170408:625,a8cc25d9-6aa0-49f1-9f45-c3958720e4ee,NaN,NaN
1488,"108810, Москва г, п. Марушкинское, д. Большое ...",6b8b5a0c-df4f-4e80-887b-76144513e162,3FD88C11-A915-4269-B216-438454B2695D,45949000,УО,1.207700e+12,773001001.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""УК Н...",Многоквартирный,Исправный,793.7,671,-,-,-,NaN,NaN,NaN,NaN,77:18:0170408:592,b25d5431-8ce0-4e34-8377-e50b17eb09d0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5160120,"129323, Москва г, проезд. Лазоревый, д. 6",689fd70f-3055-4465-8190-21ccac493d57,689FD70F-3055-4465-8190-21CCAC493D57,45361000,УО,1.087746e+12,771601001.0,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,6892.7,6892,-,-,-,NaN,NaN,NaN,NaN,77:02:0014011:1055,9d85374a-7c36-4f60-8cdb-43edc2300929,NaN,NaN
5160265,"129323, Москва г, проезд. Лазоревый, д. 8",ca25b32a-eb6b-4d25-9dc9-de2d32e321f7,CA25B32A-EB6B-4D25-9DC9-DE2D32E321F7,45361000,УО,1.087746e+12,771601001.0,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,6892.9,6892.9,-,-,-,NaN,NaN,NaN,NaN,77:02:0014011:1056,d9596c10-41ba-4fb5-9da6-5fe8b03a8fb0,NaN,NaN
5160410,"129323, Москва г, проезд. Русанова, д. 11",e61605c6-439f-46ac-8feb-fe5faf478145,E61605C6-439F-46AC-8FEB-FE5FAF478145,45361000,УО,5.167746e+12,771601001.0,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,13456.2,9437.4,-,-,-,NaN,NaN,NaN,NaN,77:02:0014011:13860,a87c74af-1983-4b25-9ae4-529044cf0fb6,NaN,NaN
5160596,"129323, Москва г, проезд. Русанова, д. 25, кор...",f87cae10-8731-4507-9184-f28ad94ee09a,F87CAE10-8731-4507-9184-F28AD94EE09A,45361000,УО,1.087746e+12,771601001.0,ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ УПРАВ...,Многоквартирный,Исправный,29049.2,21213,-,-,-,NaN,NaN,NaN,NaN,77:02:0014011:13149,21357c11-18f4-4487-8281-4324d6df4ba9,NaN,NaN


In [13]:
dmr_fias = set(prepared_dmr['N_FIAS'])
dmr_kadn = set(prepared_dmr['KAD_N'])

In [14]:
houses_data_fias = set(houses_data['Глобальный уникальный идентификатор дома по ФИАС'])
houses_data_kadn = set(houses_data['Кадастровый номер'])

In [15]:
houses_data_nr_fias = set(houses_data_not_room['Глобальный уникальный идентификатор дома по ФИАС'])
houses_data_nr_kadn = set(houses_data_not_room['Кадастровый номер'])

In [16]:
print('fias:', len(houses_data_fias), len(houses_data_nr_fias), len(dmr_fias))
print('kadn', len(houses_data_kadn), len(houses_data_nr_kadn), len(dmr_kadn))

fias: 68159 68159 420680
kadn 2872463 47094 188325


In [17]:
print(len(houses_data_fias & dmr_fias), len(houses_data_kadn & dmr_kadn))
print(len(houses_data_nr_fias & dmr_fias), len(houses_data_nr_kadn & dmr_kadn))

67262 36784
67262 36783


In [18]:
# house_data_merged = prepared_dmr.merge(
#     houses_data,
#     left_on=['N_FIAS'],
#     right_on=['Глобальный уникальный идентификатор дома по ФИАС'],
#     how='left'
#     )